#### Imports

In [1]:
import pandas as pd
import random
import numpy as np
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import itertools

#### Read Data

In [16]:
df = pd.read_csv('cleaned_train_with_features.csv')
X, y = df.drop('TARGET',axis=1), df['TARGET']

### Data Splits

#### Stratified Split

In [3]:
# Stratified Split
def stratified_split(data, test_size=0.1, validation_size = 0.1):

    class_1 = data[data['TARGET'] == 1]
    class_0 = data[data['TARGET'] == 0]

    test_count_1 = int(len(class_1) * test_size)
    validation_count_1 = int(len(class_1) * validation_size)
    test_count_0 = int(len(class_0) * test_size)
    validation_count_0 = int(len(class_0) * validation_size)

    class_1 = class_1.sample(frac=1)
    class_0 = class_0.sample(frac=1)

    # Split each class into test, validation, and train sets
    test_data = pd.concat([class_1.iloc[:test_count_1], class_0.iloc[:test_count_0]])
    validation_data = pd.concat([
        class_1.iloc[test_count_1:test_count_1 + validation_count_1],
        class_0.iloc[test_count_0:test_count_0 + validation_count_0]
    ])
    train_data = pd.concat([
        class_1.iloc[test_count_1 + validation_count_1:],
        class_0.iloc[test_count_0 + validation_count_0:]
    ])

    # Split features and target for each set
    X_train = train_data.drop('TARGET',axis=1)
    X_test = test_data.drop('TARGET',axis=1)
    X_val = validation_data.drop('TARGET',axis=1)
    y_train = train_data['TARGET']
    y_test = test_data['TARGET']
    y_val = validation_data['TARGET']

    return X_train, X_test, X_val, y_train, y_test, y_val

In [6]:
X_train, X_test, X_val, y_train, y_test, y_val = stratified_split(df)
y_val.head

<bound method NDFrame.head of 187047    1
82891     1
39875     1
212962    1
13540     1
         ..
255389    0
214840    0
136169    0
218754    0
139408    0
Name: TARGET, Length: 26341, dtype: int64>

#### Custom Split

In [4]:
# Custom Split

def custom_split(data, valid_prop = 0.1, test_prop = 0.1, random_seed = 1738):

    train_prop = 1 - valid_prop - test_prop

    # define bins for age and income
    age_bins = [0, 40, 60, np.inf]
    age_labels = ['young', 'middle_aged', 'senior']
    income_bins = [0, 30000, 70000, np.inf]
    income_labels = ['low', 'medium', 'high']

    # convert age to years
    data['age'] = data['DAYS_BIRTH']/-365

    # create binned variables
    data['age_group'] = pd.cut(data['age'], bins=age_bins, labels=age_labels)
    data['income_group'] = pd.cut(data['AMT_INCOME_TOTAL'], bins=income_bins, labels=income_labels)

    # create a key for each group (combination of gender, age, and income)
    data['group_key'] = data['CODE_GENDER_M'].astype(str) + '_' + data['age_group'].astype(str) + '_' + data['income_group'].astype(str)

    # shuffle the data
    data = data.sample(frac=1, random_state=random_seed).reset_index(drop=True)

    # split the data based on key
    train_data = pd.DataFrame()
    val_data = pd.DataFrame()
    test_data = pd.DataFrame()

    for key, group in data.groupby('group_key'):
        n = len(group)
        n_train = int(n * train_prop)
        n_val = int(n * valid_prop)

        train_data = pd.concat([train_data, group[:n_train]])
        val_data = pd.concat([val_data, group[n_train:n_train + n_val]])
        test_data = pd.concat([test_data, group[n_train + n_val:]])

    # drop all unnecesary columns
    train_data = train_data.drop(columns=['age','age_group', 'income_group', 'group_key'])
    val_data = val_data.drop(columns=['age','age_group', 'income_group', 'group_key'])
    test_data = test_data.drop(columns=['age','age_group', 'income_group', 'group_key'])

    X_train = train_data.drop('TARGET', axis=1)
    y_train = train_data['TARGET']

    X_validation = val_data.drop('TARGET', axis=1)
    y_validation = val_data['TARGET']


    X_test = test_data.drop('TARGET', errors='ignore')
    y_test = test_data['TARGET']

    return X_train, X_test, X_validation, y_train, y_test, y_validation


#### Random Split

In [5]:
def random_train_test_split(X, y, test_size=0.2, val_size=0.2, random_seed=42):
    np.random.seed(random_seed)

    n = len(X)
    indices = np.arange(n)
    np.random.shuffle(indices)

    test_indices = indices[:int(n * test_size)]
    val_indices = indices[int(n * test_size):int(n * (test_size + val_size))]
    train_indices = indices[int(n * (test_size + val_size)):]

    X_train, y_train = X.iloc[train_indices], y.iloc[train_indices]
    X_test, y_test = X.iloc[test_indices], y.iloc[test_indices]
    X_val, y_val = X.iloc[val_indices], y.iloc[val_indices]

    return X_train, X_test, X_val, y_train, y_test, y_val

#### Balanced Training Data Split

In [6]:
# custom split 2

def balanced_train_split(data, valid_prop = 0.1, test_prop = 0.1, random_seed = 1738 ):

    class_1 = data[data['TARGET'] == 1]
    class_0 = data[data['TARGET'] == 0]

    test_count_1 = int(len(class_1) * test_prop)
    validation_count_1 = int(len(class_1) * valid_prop)
    test_count_0 = int(len(class_0) * test_prop)
    validation_count_0 = int(len(class_0) * valid_prop)

    class_1 = class_1.sample(frac=1, random_state = random_seed)
    class_0 = class_0.sample(frac=1, random_state = random_seed)

    # Split each class into test, validation, and train sets
    test_data = pd.concat([class_1.iloc[:test_count_1], class_0.iloc[:test_count_0]])
    validation_data = pd.concat([
        class_1.iloc[test_count_1:test_count_1 + validation_count_1],
        class_0.iloc[test_count_0:test_count_0 + validation_count_0]
    ])
    train_data = pd.concat([
        class_1.iloc[test_count_1 + validation_count_1:],
        class_0.iloc[test_count_0 + validation_count_0:]
    ])

    # randomly remove training observations in class 0
    #num_class_1_train = len(train_data[train_data['TARGET'] == 1])
    #n_obs = int(num_class_1_train * 2)
    class_0_train = train_data[train_data['TARGET'] == 0]#.sample(n=n_obs, random_state=random_seed)
    class_1_train = train_data[train_data['TARGET'] == 1]

    num_samples = int((len(class_0_train) - len(class_1_train)) / 1.5)
    #print(num_samples)

    class_1_train_dups = train_data[train_data['TARGET'] == 1].sample(n = num_samples, replace=True, random_state=random_seed)

    balanced_train_data = pd.concat([class_0_train, class_1_train, class_1_train_dups]).sample(frac=1, random_state=random_seed)

    # Split features and target for each set
    X_train = balanced_train_data.drop('TARGET', axis=1)
    y_train = balanced_train_data['TARGET']
    X_test = test_data.drop('TARGET', axis=1)
    y_test = test_data['TARGET']
    X_val = validation_data.drop('TARGET', axis=1)
    y_val = validation_data['TARGET']

    return X_train, X_test, X_val, y_train, y_test, y_val

#### K-Fold Cross Validation

In [7]:
# Split Data Randomly Into K Folds
def kfold_cv(data, k = 5, seed = 1738):
    shuffled_data = data.sample(frac = 1, random_state = seed).reset_index(drop=True)

    n = len(shuffled_data)

    split_size = n//k
    splits = []

    for i in range(k):
        start = (i)*split_size
        end = (i+1) * split_size
        splits.append(shuffled_data.iloc[start:end])

    return splits


kfold_cv(df)

[       TARGET  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
 0           0      0.000000          0.001316    0.078652     0.053686   
 1           0      0.000000          0.000835    0.052360     0.049667   
 2           0      0.052632          0.003239    0.240180     0.176185   
 3           0      0.052632          0.000469    0.310449     0.140611   
 4           0      0.000000          0.001700    0.157303     0.125325   
 ...       ...           ...               ...         ...          ...   
 52678       0      0.000000          0.001239    0.158427     0.079695   
 52679       0      0.000000          0.000508    0.012719     0.030133   
 52680       0      0.052632          0.000816    0.079757     0.033292   
 52681       0      0.052632          0.000546    0.061073     0.051860   
 52682       0      0.052632          0.000739    0.177326     0.130677   
 
        AMT_GOODS_PRICE  REGION_POPULATION_RELATIVE  DAYS_BIRTH  DAYS_EMPLOYED  \
 0             0

### Metrics

#### Recall

In [8]:
def recall(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    TP = np.sum((y_true == 1) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))
    
    return float(TP / (TP + FN)) if (TP + FN) > 0 else 0.0

#### Precision

In [9]:
def precision(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    TP = np.sum((y_true == 1) & (y_pred == 1))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    
    return float(TP / (TP + FP)) if (TP + FP) > 0 else 0.0

#### Accuracy

In [10]:
# Accuracy
def compute_accuracy(truth, predicted):
    return np.mean(truth == predicted)


#### F1 Score

In [11]:
# F1 Score
def calculate_f1(y_true, y_pred):
    TP = FP = TN = FN = 0

    for true, pred in zip(y_true, y_pred):
        if true == 1 and pred == 1:
            TP += 1
        elif true == 0 and pred == 1:
            FP += 1
        elif true == 0 and pred == 0:
            TN += 1
        elif true == 1 and pred == 0:
            FN += 1 

    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    if precision + recall == 0:
        return 0

    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

#### F2 Score

In [12]:
# F2 Score
def calculate_f2(y_true, y_pred, b=2):
    r = recall(y_true, y_pred)
    p = precision(y_true, y_pred)
    
    if r + p == 0:
        return 0
    
    f2 = (1 + b**2) * (r * p) / (b**2 * r + p)
    return f2

#### ROC-AUC

In [13]:
def calc_roc_auc(y_true, y_prob):
  sorted_indices = np.argsort(y_prob)
  true_sort = y_true[sorted_indices]
  prob_sort = y_prob[sorted_indices]

  TP = np.cumsum(true_sort)
  FP = np.cumsum(1 - true_sort)

  TPR = TP / TP[-1]
  FPR = FP / FP[-1]

  

  # rocauc
  rocauc_score = np.trapz(FPR, TPR)

  return rocauc_score

### Fairness Metric

In [14]:
# Equal Opportunity
def equal_opportunity(classes,truth,preds):
    df = classes
    df["truth"] = truth.values
    df_preds = pd.DataFrame(preds, columns=["preds"])
    df = df.join(df_preds)
    scores = {}

    for key,group in df.groupby(['CODE_GENDER_M', 'AGE_GROUP']):
        scores[key] = recall(group['truth'], group['preds'])

    values = list(scores.values())
    diffs = []

    for pair in itertools.combinations(values, 2):
        diffs.append(abs(pair[0] - pair[1]))

    average_difference = sum(diffs) / len(diffs)


    return scores, average_difference

# testing
age_bins = [0, 40, 60, np.inf]
age_labels = ['young', 'middle_aged', 'senior']

df['age'] = df['DAYS_BIRTH']/-365

df['AGE_GROUP'] = pd.cut(df['age'], bins=age_bins, labels=age_labels)

ex_classes = df[['CODE_GENDER_M', 'AGE_GROUP']]

#preds = lda_model.predict(X)

#df_preds = pd.DataFrame(preds, columns=["preds"])
#classes.join(df_preds)
#print(equal_opportunity(ex_classes,y,preds)[0], "\n", equal_opportunity(ex_classes,y,preds)[1])

#unique_values, counts = np.unique(preds, return_counts=True)
#print(unique_values)
#print(counts)


### Cross-Validated Metrics

In [15]:
# Perform CV and Compute all Metrics

def calc_cv_metrics(model, folds):
    num_folds = len(folds)

    metrics = {"accuracy": [],
               "precision": [],
               "recall": [],
               "f1": [],
               "roc auc": []}

    for i in range(num_folds):
        # separate folds into training and testing
        test_data = folds[i]
        train_data = pd.DataFrame()
        other_folds = folds[i + 1:] + folds[:i]
        for fold in other_folds:
            train_data = pd.concat([train_data, fold])
        
        X_train = train_data.drop('TARGET', axis=1)
        y_train = train_data['TARGET']
        X_test = test_data.drop('TARGET', axis=1)
        y_test = test_data['TARGET']
        
        # fit model and get predictions
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        probs = model.predict_proba(X_test)[:,1]

        # calculate metrics and add to dictionary
        acc = compute_accuracy(y_test, preds)
        metrics["accuracy"].append(acc)

        p = precision(y_test, preds)
        metrics["precision"].append(p)

        r = recall(y_test, preds)
        metrics["recall"].append(r)

        f1 = calculate_f1(y_test, preds) 
        metrics["f1"].append(f1)
        
        roc_auc = calc_roc_auc(np.array(y_test), probs)
        metrics["roc auc"].append(roc_auc)

    averages = {key: sum(values) / len(values) for key, values in metrics.items()}
    return averages

### Models

#### Logistic Regression

In [35]:
# logistic regression
lr = LogisticRegression(penalty=None)
X_train, X_test, X_val, y_train, y_test, y_val = stratified_split(df)
lr.fit(X_train, y_train)

preds = lr.predict(X_test)

unique_values, counts = np.unique(y_train, return_counts=True)
print(unique_values)
print(counts)

unique_values, counts = np.unique(y_test, return_counts=True)
print(unique_values)
print(counts)


unique_values, counts = np.unique(preds, return_counts=True)
print(unique_values)
print(counts)

#print(f'weights: {weights} \n constant: {b}')

# Metrics
print("precision = ", precision(y_test, preds), "\nrecall = ", recall(y_test, preds), "\nf1 score = ", calculate_f1(y_test, preds))

[0 1]
[194410  16327]
[0 1]
[24301  2040]
[0 1]
[26338     3]
precision =  0.0 
recall =  0.0 
f1 score =  0


/usr/local/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Logistic Regression W/ Penalization

In [34]:
# scale the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# logistic regression w penalty
lr_penalty = LogisticRegression(penalty='elasticnet', l1_ratio=0.5, C=1.5, solver="saga")
X_train, X_test, X_val, y_train, y_test, y_val = balanced_train_split(df)
lr_penalty.fit(X_train, y_train)

preds = lr_penalty.predict(X_test)

# Evaluate predictio
unique_values, counts = np.unique(y_train, return_counts=True)
print(unique_values)
print(counts)

unique_values, counts = np.unique(y_test, return_counts=True)
print(unique_values)
print(counts)


unique_values, counts = np.unique(preds, return_counts=True)
print(unique_values)
print(counts)

#print(f'weights: {weights} \n constant: {b}')

# Metrics
print("precision = ", precision(y_test, preds), "\nrecall = ", recall(y_test, preds), "\nf1 score = ", calculate_f1(y_test, preds))

[0 1]
[32654 16327]
[0 1]
[24301  2040]
[0 1]
[23338  3003]
precision =  0.2077922077922078 
recall =  0.3058823529411765 
f1 score =  0.24747174301011302


/usr/local/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


#### SVC

In [68]:
# SVC Model
svc_model = SVC(gamma='auto', kernel='linear') #Can add standard scaler if you want to scale, but left it out to interpret results
svc_model.fit(X, y)
weights = svc_model.coef_
b = svc_model.intercept_
print(f'weights: {weights} \n constant: {b}')
print(f'Actual decision boundary: {-b/weights}')


#### LDA


In [66]:
# LDA Model
#print(df)
lda_model = LinearDiscriminantAnalysis()

X_train, X_test, X_val, y_train, y_test, y_val = random_train_test_split(X,y)


lda_model.fit(X_train,y_train)

preds = lda_model.predict(X_test)
probabilities = lda_model.predict_proba(X_test)
print(probabilities)
print(preds)

unique_values, counts = np.unique(y_train, return_counts=True)
print(unique_values)
print(counts)

unique_values, counts = np.unique(y_test, return_counts=True)
print(unique_values)
print(counts)


unique_values, counts = np.unique(preds, return_counts=True)
print(unique_values)
print(counts)

#print(f'weights: {weights} \n constant: {b}')
#print(y_test)
# Metrics
print("precision = ", precision(y_test, preds), "\nrecall = ", recall(y_test, preds), "\nf1 score = ", calculate_f1(y_test, preds), "\nROC AUC =", calc_roc_auc(np.array(y_test),probabilities[:,1]))

[[0.94819732 0.05180268]
 [0.99169357 0.00830643]
 [0.92519881 0.07480119]
 ...
 [0.84224421 0.15775579]
 [0.89670206 0.10329794]
 [0.87179546 0.12820454]]
[0 0 0 ... 0 0 0]
[0 1]
[145748  12304]
[0 1]
[48644  4039]
[0 1]
[52646    37]
precision =  0.3783783783783784 
recall =  0.0034662045060658577 
f1 score =  0.006869479882237487 
ROC AUC = 0.7072990637558778


# Cross Validation For Model Selection (all features)

### LDA

In [67]:
# LDA

# Balanced Training Data Split
# The test data in our Cross validation folds does not represent our test data, so we will use the validation set for balanced train split
X_train, X_test, X_val, y_train, y_test, y_val = balanced_train_split(df)
train_data = X_train.join(y_train)
lda = LinearDiscriminantAnalysis()

lda.fit(X_train,y_train)

preds = lda.predict(X_test)
probabilities = lda.predict_proba(X_test)

# Metrics
metrics = {'accuracy': compute_accuracy(y_test,preds),
           "precision": precision(y_test, preds),
           "recall": recall(y_test, preds),
           "f1": calculate_f1(y_test, preds),
           "roc auc": calc_roc_auc(np.array(y_test),probabilities[:,1])}
           
print("balanced training split: ", metrics)

# Random split
X_train, X_test, X_val, y_train, y_test, y_val = random_train_test_split(X,y)
train_data = X_train.join(y_train)

folds = kfold_cv(train_data)

print("random split: ", calc_cv_metrics(lda, folds))

# Stratified split
X_train, X_test, X_val, y_train, y_test, y_val = stratified_split(df)
train_data = X_train.join(y_train)

folds = kfold_cv(train_data)

print("stratified split: ", calc_cv_metrics(lda, folds))

# Custom Split (stratified based on income, age, and gender)
X_train, X_test, X_val, y_train, y_test, y_val = stratified_split(df)
train_data = X_train.join(y_train)

folds = kfold_cv(train_data)

print("fair? split: ", calc_cv_metrics(lda, folds))


balanced training split:  {'accuracy': 0.7814433772445997, 'precision': 0.16901157613535173, 'recall': 0.46519607843137256, 'f1': 0.24794252122795557, 'roc auc': 0.705172969562295}
random split:  {'accuracy': 0.9219614046187914, 'precision': 0.3627427040771761, 'recall': 0.0034989461609636965, 'f1': 0.006927915373608863, 'roc auc': 0.707719535382512}
stratified split:  {'accuracy': 0.9223954255344389, 'precision': 0.4190787313110362, 'recall': 0.0038628101333784796, 'f1': 0.007652225707319715, 'roc auc': 0.707399424389586}
fair? split:  {'accuracy': 0.9223622084608631, 'precision': 0.39187695924764887, 'recall': 0.003792993301911696, 'f1': 0.007511206420255744, 'roc auc': 0.7064893083601604}


## Logistic Regression

In [69]:
# Logistic Regression With No Penalty

# Balanced Training Data Split
# The test data in our Cross validation folds does not represent our test data, so we will use the validation set for balanced train split
X_train, X_test, X_val, y_train, y_test, y_val = balanced_train_split(df)
train_data = X_train.join(y_train)
logit = LogisticRegression(penalty=None, max_iter=1000)

logit.fit(X_train,y_train)

preds = logit.predict(X_test)
probabilities = logit.predict_proba(X_test)

# Metrics
metrics = {'accuracy': compute_accuracy(y_test,preds),
           "precision": precision(y_test, preds),
           "recall": recall(y_test, preds),
           "f1": calculate_f1(y_test, preds),
           "roc auc": calc_roc_auc(np.array(y_test),probabilities[:,1])}
print("balanced training split: ", metrics)

# Random split
X_train, X_test, X_val, y_train, y_test, y_val = random_train_test_split(X,y)
train_data = X_train.join(y_train)

folds = kfold_cv(train_data)

print("random split: ", calc_cv_metrics(logit, folds))

# Stratified split
X_train, X_test, X_val, y_train, y_test, y_val = stratified_split(df)
train_data = X_train.join(y_train)

folds = kfold_cv(train_data)

print("stratified split: ", calc_cv_metrics(logit, folds))

# Custom Split (stratified based on income, age, and gender)
X_train, X_test, X_val, y_train, y_test, y_val = stratified_split(df)
train_data = X_train.join(y_train)

folds = kfold_cv(train_data)

print("fair? split: ", calc_cv_metrics(logit, folds))


balanced training split:  {'accuracy': 0.7817091226604912, 'precision': 0.17004624688722875, 'recall': 0.46862745098039216, 'f1': 0.24954320020882279, 'roc auc': 0.7056903976355366}
random split:  {'accuracy': 0.9221195824106294, 'precision': 0.4238095238095238, 'recall': 0.001301054061712379, 'f1': 0.0025926862620463936, 'roc auc': 0.7061998997679244}
stratified split:  {'accuracy': 0.9224950767551665, 'precision': 0.3658119658119658, 'recall': 0.000985878714079184, 'f1': 0.001966052891926605, 'roc auc': 0.7054985794044194}
fair? split:  {'accuracy': 0.9225045673476167, 'precision': 0.4587301587301587, 'recall': 0.0008554201913810738, 'f1': 0.001707237434595034, 'roc auc': 0.7053467103821566}


### Logistic Regression (Grid Search Across Penalty/Mixture)

In [17]:
mixture = [0,0.25,0.5,0.75,1]
penalty = [0.1,1,10,100,1000,10000]

def logit_grid_search(train_x, train_y, val_x, val_y, mix_list, penalty_list, metric = "roc auc", k = 5):

    results = []

    for m in mix_list:
        for p in penalty_list:
            #specify model
            logit = LogisticRegression(penalty='elasticnet',
                                       l1_ratio=m,
                                       C=p,
                                       max_iter=500,
                                       solver='saga')
            
            # fit on train data
            logit.fit(train_x,train_y)

            # predictions
            preds = logit.predict(val_x)
            probabilities = logit.predict_proba(val_x)

            # Metrics
            new_row = {'mixture': m,
                       'penalty': p,
                       'accuracy': compute_accuracy(val_y,preds),
                       "precision": precision(val_y, preds),
                       "recall": recall(val_y, preds),
                       "f1": calculate_f1(val_y, preds),
                       "roc auc": calc_roc_auc(np.array(val_y),probabilities[:,1])}
            print(new_row)
            results.append(new_row)

    df = pd.DataFrame(results)
    sorted_df = df.sort_values(by = metric, ascending= False)

    return sorted_df
    

#X_train, X_test, X_val, y_train, y_test, y_val = balanced_train_split(df)
#logit_grid_search(X_train, y_train, X_val, y_val, mixture, penalty)

            




### SVC

# FEATURE SELECTION (top 50 coefficients from logistic regression with L1 penalty)

**Penalty Selected by Grid Search**

In [18]:

mixture = [1]
penalty = [0.01,0.1,1,10,100,1000]

X_train, X_test, X_val, y_train, y_test, y_val = balanced_train_split(df)
logit_grid_search(X_train, y_train, X_val, y_val, mixture, penalty)


{'mixture': 1, 'penalty': 0.01, 'accuracy': 0.7861508674689648, 'precision': 0.1668829964769145, 'recall': 0.4411764705882353, 'f1': 0.24216332570967308, 'roc auc': 0.6975690502529146}
{'mixture': 1, 'penalty': 0.1, 'accuracy': 0.7856953038988649, 'precision': 0.17077625570776256, 'recall': 0.4583333333333333, 'f1': 0.2488356620093147, 'roc auc': 0.7087672499558237}


/usr/local/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


{'mixture': 1, 'penalty': 1, 'accuracy': 0.7837971223567822, 'precision': 0.171017101710171, 'recall': 0.46568627450980393, 'f1': 0.25016458196181696, 'roc auc': 0.7089481309169073}


/usr/local/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


{'mixture': 1, 'penalty': 10, 'accuracy': 0.7833415587866823, 'precision': 0.17088493986716927, 'recall': 0.4666666666666667, 'f1': 0.2501642359742478, 'roc auc': 0.7089648130352095}


/usr/local/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


{'mixture': 1, 'penalty': 100, 'accuracy': 0.7832276678941574, 'precision': 0.1707929673484033, 'recall': 0.4666666666666667, 'f1': 0.2500656685053848, 'roc auc': 0.7089779650801105}
{'mixture': 1, 'penalty': 1000, 'accuracy': 0.7832656315249991, 'precision': 0.17082361385250314, 'recall': 0.4666666666666667, 'f1': 0.25009851569683433, 'roc auc': 0.7089773397528223}


/usr/local/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


,mixture,penalty,accuracy,precision,recall,f1,roc auc
4,1,100.00,0.783228,0.170793,0.466667,0.250066,0.708978
5,1,1000.00,0.783266,0.170824,0.466667,0.250099,0.708977
3,1,10.00,0.783342,0.170885,0.466667,0.250164,0.708965
2,1,1.00,0.783797,0.171017,0.465686,0.250165,0.708948
1,1,0.10,0.785695,0.170776,0.458333,0.248836,0.708767
0,1,0.01,0.786151,0.166883,0.441176,0.242163,0.697569


#### Fit Model To Find Top Coefficients
Best Penalty: 1 (almost equal roc auc, best f1)

In [31]:

X_train, X_test, X_val, y_train, y_test, y_val = balanced_train_split(df)
logit = LogisticRegression(penalty='l1', C = 1, solver='saga')

logit.fit(X_train, y_train)


feature_names = X_train.columns

# Get the coefficients from your logistic regression model
#intercept = logit.intercept_[0]
coefficients = logit.coef_.flatten()

# Combine them into a DataFrame for easier viewing
coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# Sort by the absolute value of the coefficients (optional)
coef_df['Abs_Coefficient'] = coef_df['Coefficient'].abs()
coef_df = coef_df.sort_values(by='Abs_Coefficient', ascending=False).head(30)

keep_features = coef_df['Feature']

target = pd.Series(["TARGET"], index=[len(keep_features)])

# Concatenate 'TARGET' to the existing Series
keep_features = pd.concat([keep_features, target])

print(keep_features)



150                                ORGANIZATION_TYPE_XNA
7                                          DAYS_EMPLOYED
155                                       EMPLOYED_YEARS
152                                    INCOME_PER_PERSON
4                                        AMT_GOODS_PRICE
2                                             AMT_CREDIT
10                                            FLAG_MOBIL
56                             AMT_REQ_CREDIT_BUREAU_QRT
74                              NAME_INCOME_TYPE_Working
68                 NAME_INCOME_TYPE_Commercial associate
71                        NAME_INCOME_TYPE_State servant
28                              DEF_30_CNT_SOCIAL_CIRCLE
70                            NAME_INCOME_TYPE_Pensioner
73                           NAME_INCOME_TYPE_Unemployed
47                                      FLAG_DOCUMENT_17
118                   ORGANIZATION_TYPE_Industry: type 8
1                                       AMT_INCOME_TOTAL
26                             

/usr/local/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


#### CREATE NEW DF WITH ONLY 30 FEATURES

In [32]:
reduced_df = df[keep_features]

print(reduced_df)

        ORGANIZATION_TYPE_XNA  DAYS_EMPLOYED  EMPLOYED_YEARS  \
0                           0       0.954914        0.954914   
1                           0       0.956352        0.956352   
2                           0       0.953839        0.953839   
3                           0       0.961180        0.961180   
4                           0       0.957396        0.957396   
...                       ...            ...             ...   
263414                      0       0.954293        0.954293   
263415                      1       0.000000        0.000000   
263416                      0       0.973924        0.973924   
263417                      0       0.965742        0.965742   
263418                      0       0.956545        0.956545   

        INCOME_PER_PERSON  AMT_GOODS_PRICE  AMT_CREDIT  FLAG_MOBIL  \
0                0.005121         0.077441    0.090287           1   
1                0.003390         0.271605    0.311736           1   
2                0.00